In [30]:
# imports 
import json
import multiprocessing 
import os
import re
import string
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

# from gensim.corpora import Dictionary

from datahandler import DataHandler

In [11]:
# fcns

stopwords = nltk.corpus.stopwords.words()


def filter_ngram(ngram, n:int):
    tag = nltk.pos_tag(ngram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if n == 2:
        if ngram[0] in stopwords or ngram[1] in stopwords:
            return False
    if n==3:
        if ngram[0] in stopwords or ngram[-1] in stopwords or ngram[1] in stopwords:
            return False
    if 'n' in ngram or 't' in ngram:
        return False
    if 'PRON' in ngram:
        return False
    return True


def merge_ngram(x, bigrams, trigrams):
    for gram in trigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    return x
        

def filter_stopwords(x):
    return [word for word in x.split() if word not in stopwords and len(word)>2]


def filter_pos(x):
    pos = nltk.pos_tag(x)
    filtered = [word[0] for word in pos if word[1] in ['NN']]
    return filtered


In [12]:
seed = 123
data_dir = os.path.join(os.pardir, os.pardir, "web_data", "preproc")
print("Loading corpus")
corpus = DataHandler(data_dir, seed)

# print some various information from the corpus
print("Total Word Count: {}".format(corpus.total_words))
print("Number of Docs in the Corpus: {}".format(corpus.total_docs))

docs_fpath = corpus.data.keys()

# create dictionary for filename and text
fpath_txt = {}
for fpath in docs_fpath:
    with open(fpath, "r") as f:
        fpath_txt[fpath] = f.read()

# make dataframe
df = (pd.DataFrame.from_dict(fpath_txt, orient='index')
         .reset_index().rename(index = str, columns = {'index': 'file_name', 0: 'text'}))

corpus = df['text']
print("Finished loading corpus")


Loading corpus
Total Word Count: 2220710
Number of Docs in the Corpus: 5405
Finished loading corpus


In [13]:
min_bigram_frequency = 50

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents([doc.split() for doc in corpus])
finder.apply_freq_filter(min_bigram_frequency)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)

bigram_pmi = pd.DataFrame(bigram_scores)
bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)

min_trigram_frequency = 50

trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_documents([doc.split() for doc in corpus])
finder.apply_freq_filter(min_trigram_frequency)
trigram_scores = finder.score_ngrams(trigram_measures.pmi)

trigram_pmi = pd.DataFrame(trigram_scores)
trigram_pmi.columns = ['trigram', 'pmi']
trigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)

print("cell done")


cell done


In [14]:
min_pmi = 5
max_ngrams = 500

filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              filter_ngram(bigram['bigram'], 2)\
                                              and min_pmi > 5, axis = 1)][:max_ngrams]

filtered_trigram = trigram_pmi[trigram_pmi.apply(lambda trigram: \
                                                 filter_ngram(trigram['trigram'], 3)\
                                                 and min_pmi > 5, axis = 1)][:max_ngrams]


bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if len(x[0]) > 2 or len(x[1]) > 2]
trigrams = [' '.join(x) for x in filtered_trigram.trigram.values if len(x[0]) > 2 or len(x[1]) > 2 and len(x[2]) > 2]

print("cell done")


cell done


In [15]:
corpus_w_ngrams = corpus.copy()
corpus_w_ngrams = corpus_w_ngrams.map(lambda x: merge_ngram(x, bigrams, trigrams))

print("cell done")


cell done


In [16]:
p = multiprocessing.Pool()
corpus_w_ngrams = p.map(filter_stopwords, [doc for doc in corpus_w_ngrams])
p.close()
print("cell done")


cell done


In [17]:
p = multiprocessing.Pool()
final_corpus = p.map(filter_pos, [doc for doc in corpus_w_ngrams])
p.close()
print("cell done")


cell done


In [21]:
dictionary = gensim.corpora.Dictionary(final_corpus)
dictionary.filter_extremes(no_below=10, no_above=0.20)
corpus_bow = [dictionary.doc2bow(doc) for doc in final_corpus]
print("cell done")


cell done


In [22]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(corpus_bow, num_topics=5, id2word = dictionary, passes=40,\
               iterations=200, chunksize = 100, eval_every = None)
print("cell done")


cell done


In [23]:
p = pyLDAvis.gensim.prepare(ldamodel, corpus_bow, dictionary, mds='tsne')
pyLDAvis.save_html(p, 'web_lda_mp_debug.html')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      150.222153  -72.269554       1        1  28.392335
0       51.163364 -159.968018       2        1  26.621301
1      -73.144554   21.523815       3        1  23.738839
4      -66.410133 -110.606438       4        1  11.276582
3       54.417324   12.443444       5        1   9.970943, topic_info=           Term         Freq        Total Category  logprob  loglift
501      office  1823.000000  1823.000000  Default  30.0000  30.0000
168        wood  2456.000000  2456.000000  Default  29.0000  29.0000
389   community   887.000000   887.000000  Default  28.0000  28.0000
355      family  1398.000000  1398.000000  Default  27.0000  27.0000
108   furniture  1210.000000  1210.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
402     spaces,   214.849680   628.034873   Topic5  -4.9371   1.2328
272       area,   215.963098   748.800547   Topic5  -4.9319   1.0621
1400     façade   191.420097   625.299755   Topic5  -5.0525   1.1217
2582   greenery   160.694499   270.767630   Topic5  -5.2275   1.7837
128        team   157.594207   508.933261   Topic5  -5.2470   1.1332

[304 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1276      5  0.994928   ability
62        1  0.393389    access
62        3  0.606160    access
344       1  0.387967  addition
344       2  0.495663  addition
...     ...       ...       ...
138       1  0.144042      work
138       4  0.478427      work
138       5  0.377254      work
1326      4  0.995742      yard
1661      3  0.994931      year

[393 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 5, 4])

In [27]:
coherence = []
for ii in range(3,5):
    print('lda with {} topics'.format(ii))
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(corpus_bow, num_topics=ii, id2word = dictionary, passes=40,\
                   iterations=200, chunksize = 100, eval_every = None)
    print("fit model, computing coherence")
    cm = gensim.models.coherencemodel.CoherenceModel(model=ldamodel, texts=final_corpus,\
                                                     dictionary=dictionary, coherence='c_v')
    coherence.append((ii,cm.get_coherence()))
    print("generating tsne viz")
    p = pyLDAvis.gensim.prepare(ldamodel, corpus_bow, dictionary, mds='tsne')
    title = 'web_lda_mp_debug_cm_{}.html'.format(ii)
    pyLDAvis.save_html(p, title)
    print("done")

lda with 3 topics
fit model, computing coherence
generating tsne viz
done
lda with 4 topics
fit model, computing coherence
generating tsne viz
done


In [28]:
n_topics = [x[0] for x in coherence]
cm = [x[1] for x in coherence]

In [31]:
plt.plot(n_topics,cm)
plt.scatter(n_topics,cm)
plt.title('Number of Topics vs. Coherence')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence')
plt.xticks(x_val)
plt.savefig("topic_coherence.png")
plt.close()